In [ ]:
# Note: this notebook is a commented version of a notebook shared on kaggle!

In [1]:
import pandas as pd

from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
# installation: pip install iterative-stratification

In [6]:
# read data
df = pd.read_csv("/Users/fededagos/Dropbox/NLP project/data/train.csv")

# Transform categorical labels to dummy variables. Group by id. Sum over dummy. 
dfx = pd.get_dummies(df, columns=["discourse_type"]).groupby(["id"], as_index=False).sum()

# Generate name for the dummy columns
dummy_cols = [c for c in dfx.columns if c.startswith("discourse_type_") or c == "id" and c != "discourse_type_num"]
# dfx is now only the dataset with dummy columns selected: don't need to pass the data to do the splits
dfx = dfx[dummy_cols]

In [ ]:
# At this point dfx is a dataframe with ids, counting how many discourse types are present in that id!
# (Each id is a file (15594))

In [22]:
# Generate cross validation object
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Extract labels
labels = [c for c in dfx.columns if c != "id"]
dfx_labels = dfx[labels]

# Dummy kfold assignment
dfx["kfold"] = -1

# Split
for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    print(len(trn_), len(val_))
    
    # Change the value of the kfold column at the validation index to the value of the fold
    # This will tell us when to use the current entry in the validation set
    dfx.loc[val_, "kfold"] = fold

# merge back to original dataframe
df = df.merge(dfx[["id", "kfold"]], on="id", how="left")
print(df.kfold.value_counts())
df.to_csv("train_folds.csv", index=False)

12477 3117
12474 3120
12475 3119
12475 3119
12475 3119
0    28997
2    28968
3    28904
1    28737
4    28687
Name: kfold, dtype: int64
